In [4]:
import pandas as pd
import numpy as np
import random
import math
from random import shuffle 

class iNode:
    def __init__(self):
        self.input=0
        self.output=None
        self.weights=None
        
class oNode:
    def __init__(self):
        self.input=0
        self.output=None
        self.bias=None
        self.error=None
        
class innerNode:
    def __init__(self):
        self.input=0
        self.output=None
        self.bias=None
        self.weights=None
        self.error=None
        
class MLP:
    def __init__(self,inodes,n,innerlayers,bias,weight1,weight2,learning_rate):
        self.rate=learning_rate
        self.outputnode=oNode()
        self.layers=[[innerNode() for i in range(n)] for j in range(innerlayers)]
        self.layers=[[iNode() for i in range(inodes)]]+self.layers+[[self.outputnode]]
        for layer in range(len(self.layers)):
            if layer==0:
                for node in range(len(self.layers[layer])):
                    self.layers[layer][node].weights=[random.uniform(-1,1) for i in range(len(self.layers[layer+1]))]
            
            elif layer==len(self.layers)-1:
                for node in range(len(self.layers[layer])):
                    #self.layers[layer][node].weights=[weight2 for i in range(len(self.layers[layer+1]))]
                    self.layers[layer][node].bias=0        #change
                    
            else:
                for node in range(len(self.layers[layer])):
                    self.layers[layer][node].weights=[random.uniform(-1,1) for i in range(len(self.layers[layer+1]))]
                    self.layers[layer][node].bias=random.random()
                    
    def fun(self,x):
        return 1/(1+math.exp(-1*x))
    
    def derivative(self,x):
        return (x.output*(1-x.output))
    
    def classification(self,x):
        if x>0.5:
            return 1
        return 0
                    
    def train(self,ltrain,ztrain):
        number=0
        H=[]
        for input in ltrain:
            H.append([])
            for node in range(len(input)):
                self.layers[0][node].output=input[node]
            
            for layer in range(1,len(self.layers)-1):           #forward propagation
                for node in range(len(self.layers[layer])):
                    self.layers[layer][node].input=self.layers[layer][node].bias
                    for i in self.layers[layer-1]:
                        self.layers[layer][node].input+=(i.weights[node]*i.output)
                    self.layers[layer][node].output=self.fun(self.layers[layer][node].input)
                    H[-1].append(self.layers[layer][node].output)
            #self.outputnode.output=self.classification(self.outputnode.output)
            
            """self.outputnode.error=(ztrain[number]-self.outputnode.output)   #back propagation
            for layer in range(len(self.layers)-2,-1,-1):
                for node in range(len(self.layers[layer])):
                    self.layers[layer][node].error=0
                    for j in range(len(self.layers[layer+1])):
                        self.layers[layer][node].error+=(self.layers[layer][node].weights[j]*self.layers[layer+1][j].error)
                    self.layers[layer][node].error*=self.derivative(self.layers[layer][node])
                                                                    
            for i in range(0,len(self.layers)-1):
                for j in range(len(self.layers[i])):
                    for k in range(len(self.layers[i][j].weights)):
                        self.layers[i][j].weights[k]+=self.rate*self.layers[i+1][k].error*self.layers[i][j].output
                        
                    if i!=0:
                        self.layers[i][j].bias+=(self.rate*self.layers[i][j].error)"""
            number+=1
        return(H)
            
    def test(self,ltest,ztest):
        number=0
        count=0
        tp=0
        tn=0
        fp=0
        fn=0
        print(ztest)
        for input in ltest:
            for node in range(len(input)):
                self.layers[0][node].output=input[node]
            
            for layer in range(1,len(self.layers)):           #forward propagation
                for node in range(len(self.layers[layer])):
                    self.layers[layer][node].input=self.layers[layer][node].bias
                    for i in self.layers[layer-1]:
                        self.layers[layer][node].input+=(i.weights[node]*i.output)
                    t=self.layers[layer][node].input
                    self.layers[layer][node].output=self.fun(self.layers[layer][node].input)
            #print(t)
            self.outputnode.output=self.classification(t)
            #print(self.outputnode.output)
            if self.outputnode.output==ztest[number]:
                count+=1
                if ztest[number]==1:
                    tp+=1
                else:
                    tn+=1
            elif ztest[number]==1:
                fn+=1
            else:
                fp+=1
            number+=1
        print('accuracy =',count/len(ztest)*100,'%')
        #print('precision(+) =',tp/(tp+fp))
        #print('recall(+) =',tp/(tp+fn))
        #print('precision(-) =',tn/(tn+fn))
        #print('recall(-) =',tn/(tn+fp))
        
    def findB(self,H,T):
        T=np.array(T)
        T.shape=(len(T),1)
        Htran=np.transpose(H)
        prod=np.dot(H,Htran)
        row=len(prod)
        col=len(prod[0])
        lam_invmat=np.zeros((row,col),float)
        np.fill_diagonal(lam_invmat,random.random())
        if np.linalg.det(prod)!=0:
            temp=np.sum([lam_invmat,prod],axis=0)
            #print(lam_invmat)
            #print(prod)
            temp=np.linalg.inv(temp)
            B=np.dot(Htran,temp)
            B=np.dot(B,T)
        else:
            prod=np.dot(Htran,H)
            row=len(prod)
            col=len(prod[0])
            lam_invmat=np.zeros((row,col),float)
            np.fill_diagonal(lam_invmat,random.random())
            temp=np.sum([lam_invmat,prod],axis=0)
            #print(lam_invmat)
            #print(prod)
            temp=np.linalg.inv(temp)
            B=np.dot(temp,Htran)
            B=np.dot(B,T)
        print('len=',len(self.layers[-2]))
        for i in range(len(self.layers[-2])):
            self.layers[-2][i].weights=[B[i]]
        return B
        


data=pd.read_csv('C:\\Users\\Rakesh\\SC\\lab2\\SPECT.csv')
copy=data.values.tolist()
shuffle(copy)
#copy=data[:][:]
ltrain=[]
ztrain=[]
ltest=[]
ztest=[]
dict={}
c=0
n=data.iloc[:,1].count()

for i in range(n):            # dividing train and test dataset
    row=random.randint(0,len(copy)-1)
    if i<0.9*n:
        ltrain.append(copy[row][1:])
        ztrain.append(copy[row][0])
        if copy[row][0] not in dict:
            dict[copy[row][0]]=c
            c+=1
        copy.remove(copy[row])
    else:
        ltest.append(copy[row][1:])
        ztest.append(copy[row][0])
        copy.remove(copy[row])
        
for i in range(len(ztrain)): #changing categorical variables to numbers
    ztrain[i]=dict[ztrain[i]]
for i in range(len(ztest)):
    ztest[i]=dict[ztest[i]]  
inputs=len(ltrain[0])
M=MLP(inputs,5,1,1/6,1/(5*inputs),1/5,0.2)
H=M.train(ltrain,ztrain)
B=M.findB(H,ztrain)
M.test(ltest,ztest)
print(M.layers[-2][0].weights)
print(B)

len= 5
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
accuracy = 84.61538461538461 %
[array([0.10377351])]
[[ 0.10377351]
 [-0.28829142]
 [ 0.64271111]
 [ 0.51196342]
 [ 0.32789037]]


In [32]:
import numpy as np

mat1 = [[1, 6, 5],[3 ,4, 8],[2, 12, 3]]
mat2 = [[3, 4, 6],[5, 6, 7],[6,56, 7]]
  
# This will return dot product 
res = np.dot(mat2,mat1) 
  
# print resulted matrix 
print(res)

[[ 27 106  65]
 [ 37 138  94]
 [188 344 499]]


In [33]:
np.transpose(mat1)

array([[ 1,  3,  2],
       [ 6,  4, 12],
       [ 5,  8,  3]])

In [7]:
len(H[0])

5

In [8]:
len(H)

241

In [9]:
len(ztrain)

241

In [10]:
1+mat1

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [14]:
a=np.zeros((2,4),int)

In [16]:
np.fill_diagonal(a,3)

In [17]:
a

array([[3, 0, 0, 0],
       [0, 3, 0, 0]])

In [20]:
b=np.linalg.inv(mat1)

In [24]:
np.dot(mat1,b)

array([[ 1.00000000e+00,  0.00000000e+00, -5.55111512e-17],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-1.11022302e-16,  0.00000000e+00,  1.00000000e+00]])

In [25]:
a=[[2,0],[0,2]]
b=np.linalg.inv(a)

In [26]:
b

array([[0.5, 0. ],
       [0. , 0.5]])

In [27]:
np.dot(a,b)

array([[1., 0.],
       [0., 1.]])

In [28]:
np.linalg.det(a)

4.0

In [35]:
random.random()

0.48393699623313

In [39]:
np.sum([mat1,mat2],axis=0)

array([[ 4, 10, 11],
       [ 8, 10, 15],
       [ 8, 68, 10]])

In [46]:
a=np.fill_diagonal(np.zeros((3,3),int),random.random())

In [53]:
a=np.zeros((3,3),int)

In [55]:
np.fill_diagonal(a,2)

In [56]:
a

array([[2, 0, 0],
       [0, 2, 0],
       [0, 0, 2]])

In [10]:
c

2

In [22]:
len(ltrain[0])

22